# Project Notebook - Machine Learning I

### Libraries

In [ ]:
import pandas as pd
import numpy as np
import openml
from AdaBoostClassifier import AdaBoost
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, roc_curve, auc
import scipy.stats as ss
import scikit_posthocs as sp
import matplotlib.pyplot as plt

### Get the datasets to use throughout the project

Function to read a dataset from [OpenML-CC18 Curated Classification benchmark](https://www.openml.org/search?type=study&sort=tasks_included&study_type=task&id=99 "OpenML-CC18") givent its Task ID and return the corresponding Pandas Dataframe

In [ ]:
def getDataset(task_id):
    suite = openml.study.get_suite(99)
    task = openml.tasks.get_task(task_id)    
    dataset = openml.datasets.get_dataset(task.dataset_id)

    X, y, categorical_indicator, attribute_names = dataset.get_data(
        dataset_format="array", target=dataset.default_target_attribute
    )

    df = pd.DataFrame(X, columns=attribute_names)
    # convert:
    #   0 -> -1
    #   1 -> 1
    df['target'] = 2*y-1 
    # erase rows with NaN values
    df = df.dropna(how='any', axis=0)

    return df

Description of the datasets used in this project:
- Dataset about breast cancer, with 683 entries;
- Dataset about mushrooms, with 5644 entries;
- Dataset made with synthetic data, with 10880 entries;
- Dataset about the Wilt decease on plants, with 3279 entries;

In [ ]:
datasets = [
    ("Task ID = 15", getDataset(15)),
    ("Task ID = 24", getDataset(24)),
    ("Task ID = 3904", getDataset(3904)),
    ("Task ID = 146820", getDataset(146820))
]

### Performance evaluation of AdaBoost

To get a better understanding of how AdaBoost behaves with the chosen datasets,, we implemented the function `run_cv()` to be able to perform 10-fold Cross Validation on our set of algoritms, obtaining an accuracy value.

In [ ]:
def run_cv(X,y,algs,nfolds=10, means_only=False):
    results = {}
    kf = KFold(n_splits=nfolds, shuffle=True, random_state=1111)
    for algo_name, algo in algs:
        results[algo_name] = []
        for fold, (train_idx, test_idx) in enumerate(kf.split(X)):
            X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
            y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
            algo.fit(X_train, y_train, M=40)
            y_pred = algo.predict(X_test)
            results[algo_name].append(accuracy_score(y_test, y_pred))
    results_df = pd.DataFrame.from_dict(results)
    if not means_only:
        return results_df
    else:
        results_means = {}
        for algo_name, algo in algs:
            results_means[algo_name] = [np.mean(results[algo_name])]
        return pd.DataFrame.from_dict(results_means)

In [ ]:
for ds_id, ds in datasets:
    X = ds.drop(columns=['target'], axis=1)
    y = ds['target']

    algs = [("AdaBoost", AdaBoost())]
    result = run_cv(X, y, algs)
    print(f"Mean accuracy for dataset with {ds_id} in 10-fold CV: {np.mean(result['AdaBoost']):.5f}")


We also decided to implement `plot_cm()` in order to attain a confusion matrix for each of the fitted models.

In [ ]:
def plot_cm(model_fit,X_test,y_test):
    y_pred = model_fit.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot()

In [ ]:
for ds_id, ds in datasets:
    X = ds.drop(columns=['target'], axis=1)
    y = ds['target']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1111)
    ab = AdaBoost()
    ab.fit(X_train, y_train)
    plot_cm(ab, X_test, y_test)

We figured we could take another look at the algorithm's behaviour by checking how it learns when raising and lowering the train size.  
`plot_learning_curves()` was used to plot the Learning Curve of the fitted model. We ran 10 tests with training size = `tr_size` and stored all accuracy results in a line of train_scores and test_scores.   
Finally, we were able to get the mean training score and the standard deviation for each train size and plot the learning curves.

In [ ]:
def plot_learning_curves(model, X, y, title=''):
    training_size = np.linspace(0.1,0.9,10)
    train_scores = {}
    test_scores = {}
    for tr_size in training_size:
        train_scores[tr_size] = []
        test_scores[tr_size] = []
        for _ in range(10):
            X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=tr_size)
            model.fit(X_train, y_train)
            training_pred = model.predict(X_train)
            testing_pred = model.predict(X_test)
            train_scores[tr_size].append(accuracy_score(training_pred, y_train))
            test_scores[tr_size].append(accuracy_score(testing_pred, y_test))
        print(
            f"Train size: {tr_size}\n \
            Mean training score: {round(np.mean(train_scores[tr_size]), 5)}\n \
            Standard deviation: {round(np.std(train_scores[tr_size]), 5)}"
        )

    train_mean = np.array([np.mean(train_scores[tr_size]) for tr_size in training_size])
    train_std = np.array([np.std(train_scores[tr_size]) for tr_size in training_size])
    test_mean = np.array([np.mean(test_scores[tr_size]) for tr_size in training_size])
    test_std = np.array([np.std(test_scores[tr_size]) for tr_size in training_size])

    plt.figure(figsize=(8, 6))
    plt.plot(training_size, train_mean, 'o-', color='blue', label='Training score')
    plt.plot(training_size, test_mean, 'o-', color='green', label='Test score')
    plt.fill_between(training_size, train_mean - train_std, train_mean + train_std, alpha=0.1, color='blue')
    plt.fill_between(training_size, test_mean - test_std, test_mean + test_std, alpha=0.1, color='green')
    plt.xlabel('Training examples')
    plt.ylabel('Accuracy')
    plt.legend(loc='best')
    plt.title('Learning Curves '+title)
    plt.show()

In [ ]:
for ds_id, ds in datasets:
    X = ds.drop(columns=['target'], axis=1)
    y = ds['target']

    ab = AdaBoost()
    plot_learning_curves(ab, X, y, title="AdaBoost")

Since the ROC Curve plots the true positive rate (sensitivity) against the false positive rate (1-specificity) for various threshold values, it allows us to, once again, have a visual interpretation of AdaBoost's performance.  
Due to its robustness to class imbalance and threshold-independancy, we also believed the AUC Score to be a good measure of how good our chosen classifier was.  
We used `plot_roc_curve()` to acquire both.

In [ ]:
def plot_roc_curve(y_true, y_pred):
    fpr, tpr, _ = roc_curve(y_true, y_pred)
    roc_auc = auc(fpr, tpr)

    plt.figure()
    plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (AUC = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic')
    plt.legend(loc="lower right")
    plt.show()

In [ ]:
for ds_id, ds in datasets:
    ada_boost = AdaBoost()
    X = ds.drop(columns=['target'], axis=1)
    y = ds['target']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1111)
    ada_boost.fit(X_train, y_train)
    y_pred = ada_boost.predict(X_test)
    plot_roc_curve(y_test, y_pred)

### Comparison of base algorithm and modified ones

Definition of the models:
- `default` is the default AdaBoost classifier.
- `Alpha Changed` is the AdaBoost classifier with a different method to calculate alpha, the value used when updating the sample weights. In this model, the alpha value is directly proportional to the error - in other words, alpha == error.
- `Duplicate Misclassified` is the AdaBoost classifier that, at each iteration, duplicates misclassified samples with a given probability P. P is calculated by dividing the number of new misclassified samples by the total size of samples (including previously added ones).
- `Alpha Changed & Duplicate Misclassified` combines both of the two previous methods.

In [ ]:
models = [
    ("default", AdaBoost()),
    ("Alpha Changed", AdaBoost(alpha_type=1)),
    ("Duplicate Misclassified", AdaBoost(duplicate=True)),
    ("Alpha Changed & Duplicate Misclassified", AdaBoost(alpha_type=1, duplicate=True))
]

`plot_cv` plots the results from 10-fold Cross Validation for the 4 used algorithms. We applied this function to each dataset.

In [ ]:
def plot_cv(results_cv,metric='Accuracy', title="Cross-validation results for multiple algorithms in a single task"):
    fig, ax = plt.subplots()
    ax.boxplot(results_cv)
    ax.set_xticklabels(results_cv.columns)
    ax.set_ylabel(metric)
    ax.set_title(title)
    plt.show()

In [ ]:
for ds_id, ds in datasets:
    X = ds.drop(columns=['target'], axis=1)
    y = ds['target']
    results = run_cv(X, y, models)
    plot_cv(results, title=f"Cross-validation for the algorithms in the dataset with {ds_id}")

To have a better understanding of how effective our changes were, it's possible to see in a table below the accuracy obtained from all of the datasets with all of the new models (using the previously mentioned `run_cv` function).

In [ ]:
def run_cv_per_dataset(datasets, algs):
    table = pd.DataFrame()
    for ds_id, ds in datasets:
        X = ds.drop(columns=['target'], axis=1)
        y = ds['target']
        result_ds = run_cv(X, y, algs, means_only=True)
        table = pd.concat([table, result_ds], ignore_index=True)
    return table

In [ ]:
table = run_cv_per_dataset(datasets, models)
table

It's also possible to analyse their performance compared to each other by ranking the models. Below there's another table where each line (0-3) is a dataset and the respective collumn shows the models' position in ranking.  
Line 4 holds the average rank for each classifier.

In [ ]:
# Use the rank method to get the ranking of classifiers per data set
def get_ranks(estimates_df,asc=False):
    results_rank = estimates_df.rank(axis=1, ascending=asc, method='min')
    results_rank.loc[len(results_rank)] = results_rank.mean()
    return results_rank

In [ ]:
table_ranks = get_ranks(table)
table_ranks

Finally, we ran a Friedman test to check whether all of the models could be considered equivalent (and had equal rankings across the tasks).  
Considering that the p-value is greater than alpha, we can accept the hypothesis of the algorithms being equivalent with a significancy level of 5%.

In [ ]:
def multiple_comp(estimates_df,alpha=0.05):
    fvalue, pvalue = ss.friedmanchisquare(*[estimates_df[alg] for alg in estimates_df])
    print(f'p-value = {pvalue:.3f}')
    if pvalue >= alpha:
        print(f'There is no significant difference in performance among the algorithms')
        return None

    print(f'There is a significant difference in performance among the algorithms')
    pairedcomp = sp.posthoc_nemenyi_friedman(estimates_df)
    return(pairedcomp) 

In [ ]:
paired_comparision = multiple_comp(table)